In [31]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torcheval.metrics.functional import multiclass_accuracy
from torch.utils.data import TensorDataset, DataLoader, Subset
import random

In [2]:
rating_df = pd.read_csv('data/MovieLens/rating.csv')
rating_df.head()

,userId,movieId,rating,timestamp
0,1,2,3.5,2005-04-02 23:53:47
1,1,29,3.5,2005-04-02 23:31:16
2,1,32,3.5,2005-04-02 23:33:39
3,1,47,3.5,2005-04-02 23:32:07
4,1,50,3.5,2005-04-02 23:29:40


In [3]:
# Let's make it multi-class classification
rating_df['normalized_rating'] = rating_df.rating
rating_values = np.array(list(sorted(set(rating_df.rating))))
# num_classes = len(set(rating_df.rating))
rating_df['rating_class'] = (rating_df.rating * 2).astype(int)

In [4]:
set(rating_df.rating_class)

{1, 2, 3, 4, 5, 6, 7, 8, 9, 10}

In [5]:
subset_ratio = 0.003
rows = rating_df.userId - min(rating_df.userId) # convert into index starting at 0
cols = rating_df.movieId - min(rating_df.movieId) # convert into index starting at 0
user_movie_coo_tensor = torch.sparse_coo_tensor([rows, cols], rating_df.rating_class.to_numpy(), dtype=torch.float)
user_movie_dataset = TensorDataset(user_movie_coo_tensor.to_dense())
user_movie_dataset_length = len(user_movie_dataset)
user_movie_subset_sample_idx = random.sample(range(user_movie_dataset_length), int(user_movie_dataset_length * subset_ratio))
user_movie_subset = Subset(user_movie_dataset, user_movie_subset_sample_idx)
user_movie_dataloader = DataLoader(user_movie_subset, batch_size=64, shuffle=True)

In [6]:
class MovieLensVAE(nn.Module):
    def __init__(self, input_dim, num_classes=6, hidden_dim=600, latent_dim=200):
        super().__init__()
        self.input_dim = input_dim
        self.num_classes = num_classes
        self.encoder = nn.Sequential(
            nn.Linear(input_dim, hidden_dim, bias=True),
            nn.BatchNorm1d(hidden_dim),
            nn.ReLU()
        )
        self.mu = nn.Linear(hidden_dim, latent_dim, bias=True)
        self.logvar = nn.Linear(hidden_dim, latent_dim, bias=True)
        self.decoder = nn.Sequential(
            nn.Linear(latent_dim, hidden_dim, bias=True),
            nn.BatchNorm1d(hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, input_dim * num_classes, bias=True)
        )
    
    def forward(self, x):
        latent = self.encoder(x)
        mu = self.mu(latent)
        logvar = self.logvar(latent)
        z = self.reparameterization(mu, logvar)
        x_output = self.decoder(z)
        logits = x_output.view(-1, self.num_classes, self.input_dim)
        return logits, mu, logvar

    def reparameterization(self, mu, logvar):
        std = torch.exp(logvar/2)
        e = torch.randn_like(std)
        return mu + std * e

In [7]:
# Minimize cross entropy reconstruction_loss + KL Divergence
def loss_func(x, vae_logits, mu, logvar):
    # Reconstruction Loss
    # input = batch_size x C x movie
    # target = batch_size x movie
    reconstruction_loss = F.cross_entropy(vae_logits, x.long())
    # KL Divergence between Q(z|X) and N(0, 1)
    KLD = 0.5 * torch.mean(mu.pow(2) + logvar.exp() - logvar - 1)
    return reconstruction_loss + KLD

In [8]:
input_dim = max(cols) + 1
num_classes = len(set(rating_df.rating_class)) + 1
movielens_var = MovieLensVAE(input_dim, num_classes, hidden_dim=20, latent_dim=5)
optimizer = optim.Adam(movielens_var.parameters(), lr=1e-3)
epochs = 10

cur_mu, cur_logvar = 0, 0
for epoch in range(epochs):
    loss_err = 0
    for i, dl in enumerate(user_movie_dataloader):
        x_output, mu, logvar = movielens_var(dl[0])
        cur_mu, cur_logvar = mu, logvar
        loss = loss_func(dl[0], x_output, mu, logvar)
        loss_err += loss.item()

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    
        print(f"Epoch={epoch}, i={i}: Error={loss_err}")

Epoch=0, i=0: Error=2.558574676513672
Epoch=0, i=1: Error=5.119415283203125
Epoch=0, i=2: Error=7.621723413467407
Epoch=0, i=3: Error=10.103929281234741
Epoch=0, i=4: Error=12.565549373626709
Epoch=0, i=5: Error=15.003506422042847
Epoch=0, i=6: Error=17.42848777770996
Epoch=1, i=0: Error=2.421232223510742
Epoch=1, i=1: Error=4.815320253372192
Epoch=1, i=2: Error=7.189221620559692
Epoch=1, i=3: Error=9.538819313049316
Epoch=1, i=4: Error=11.879055261611938
Epoch=1, i=5: Error=14.198310136795044
Epoch=1, i=6: Error=16.50919246673584
Epoch=2, i=0: Error=2.282667875289917
Epoch=2, i=1: Error=4.552778959274292
Epoch=2, i=2: Error=6.804627180099487
Epoch=2, i=3: Error=9.045409917831421
Epoch=2, i=4: Error=11.254660606384277
Epoch=2, i=5: Error=13.459369897842407
Epoch=2, i=6: Error=15.65657114982605
Epoch=3, i=0: Error=2.1679744720458984
Epoch=3, i=1: Error=4.322652101516724
Epoch=3, i=2: Error=6.459149599075317
Epoch=3, i=3: Error=8.576249599456787
Epoch=3, i=4: Error=10.685420751571655
Epo

In [26]:
def predict(model, x):
    # Set the model setting to eval() mode
    model.eval()
    
    # Convert x to tensor if needed
    if not isinstance(x, torch.Tensor):
        x = torch.tensor(x, type=torch.float)
    
    with torch.no_grad():
        logits, _, _ = model(x)
        x_output = torch.softmax(logits, dim=-1)
        x_output = x_output.view(len(x), -1, num_classes)
        x_output = torch.max(x_output, dim=-1)[1]
    
    return x_output

In [34]:
for x in user_movie_dataloader:
    predicted_x = predict(movielens_var, x[0])
    acc = 0
    for batch in range(64):
        acc += multiclass_accuracy(x[0][i], predicted_x[i])
    print(acc / 64 * 100)
    break

tensor(8.9615)
